<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

In the most general case, a censored data set may contain values that
are left-censored, right-censored, interval-censored, and uncensored.
For example, here we create a data set with five observations.  Two
are uncensored (values 1 and 1.5), one is a left-censored observation
of 0, one is a right-censored observation of 10 and one is
interval-censored in the interval [2, 3].


In [ ]:
import numpy as np
from scipy.stats import CensoredData
data = CensoredData(uncensored=[1, 1.5], left=[0], right=[10],
                    interval=[[2, 3]])
print(data)

CensoredData(5 values: 2 not censored, 1 left-censored,
1 right-censored, 1 interval-censored)

Equivalently,


In [ ]:
data = CensoredData(interval=[[1, 1],
                              [1.5, 1.5],
                              [-np.inf, 0],
                              [10, np.inf],
                              [2, 3]])
print(data)

CensoredData(5 values: 2 not censored, 1 left-censored,
1 right-censored, 1 interval-censored)

A common case is to have a mix of uncensored observations and censored
observations that are all right-censored (or all left-censored). For
example, consider an experiment in which six devices are started at
various times and left running until they fail.  Assume that time is
measured in hours, and the experiment is stopped after 30 hours, even
if all the devices have not failed by that time.  We might end up with
data such as this
```

Device  Start-time  Fail-time  Time-to-failure
1         0         13           13
2         2         24           22
3         5         22           17
4         8         23           15
5        10        ***          >20
6        12        ***          >18

```
Two of the devices had not failed when the experiment was stopped;
the observations of the time-to-failure for these two devices are
right-censored.  We can represent this data with


In [ ]:
data = CensoredData(uncensored=[13, 22, 17, 15], right=[20, 18])
print(data)

CensoredData(6 values: 4 not censored, 2 right-censored)

Alternatively, we can use the method `CensoredData.right_censored` to
create a representation of this data.  The time-to-failure observations
are put the list ``ttf``.  The ``censored`` list indicates which values
in ``ttf`` are censored.


In [ ]:
ttf = [13, 22, 17, 15, 20, 18]
censored = [False, False, False, False, True, True]

Pass these lists to `CensoredData.right_censored` to create an
instance of `CensoredData`.


In [ ]:
data = CensoredData.right_censored(ttf, censored)
print(data)

CensoredData(6 values: 4 not censored, 2 right-censored)

If the input data is interval censored and already stored in two
arrays, one holding the low end of the intervals and another
holding the high ends, the class method ``interval_censored`` can
be used to create the `CensoredData` instance.

This example creates an instance with four interval-censored values.
The intervals are [10, 11], [0.5, 1], [2, 3], and [12.5, 13.5].


In [ ]:
a = [10, 0.5, 2, 12.5]  # Low ends of the intervals
b = [11, 1.0, 3, 13.5]  # High ends of the intervals
data = CensoredData.interval_censored(low=a, high=b)
print(data)

CensoredData(4 values: 0 not censored, 4 interval-censored)

Finally, we create and censor some data from the `weibull_min`
distribution, and then fit `weibull_min` to that data. We'll assume
that the location parameter is known to be 0.


In [ ]:
from scipy.stats import weibull_min
rng = np.random.default_rng()

Create the random data set.


In [ ]:
x = weibull_min.rvs(2.5, loc=0, scale=30, size=250, random_state=rng)
x[x > 40] = 40  # Right-censor values greater or equal to 40.

Create the `CensoredData` instance with the `right_censored` method.
The censored values are those where the value is 40.


In [ ]:
data = CensoredData.right_censored(x, x == 40)
print(data)

CensoredData(250 values: 215 not censored, 35 right-censored)

35 values have been right-censored.

Fit `weibull_min` to the censored data.  We expect to shape and scale
to be approximately 2.5 and 30, respectively.


In [ ]:
weibull_min.fit(data, floc=0)

(2.3575922823897315, 0, 30.40650074451254)